In [7]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime

dataDir = "/code/archive/garbage_classification/Garbage_classification"

imgHeight = 224
imgWidth = 224
batchSize = 32

trainDs = tf.keras.utils.image_dataset_from_directory(
  dataDir,
  validation_split=0.2,
  labels='inferred'
  subset="training",
  seed=123,
  image_size=(imgHeight, imgWidth),
  batch_size=batchSize)

valDs = tf.keras.utils.image_dataset_from_directory(
  dataDir,
  validation_split=0.2,
  labels='inferred'
  subset="validation",
  seed=123,
  image_size=(imgHeight, imgWidth),
  batch_size=batchSize)

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"

feature_extractor_model = mobilenet_v2

IMAGE_SHAPE = (imgHeight, imgWidth)

# Apply normalization to images
normalization_layer = tf.keras.layers.Rescaling(1./255)
trainDs = trainDs.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
valDs = valDs.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

AUTOTUNE = tf.data.AUTOTUNE
trainDs = trainDs.cache().prefetch(buffer_size=AUTOTUNE)
valDs = valDs.cache().prefetch(buffer_size=AUTOTUNE)

resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.Resizing(imgHeight,imgWidth),
])

feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(imgHeight, imgWidth, 3),
    trainable=False)

feature_extractor_layer.trainable = False


model = tf.keras.Sequential([
  resize_and_rescale,
  data_augmentation,
  feature_extractor_layer,
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])
# model.summary()




Found 2390 files belonging to 5 classes.
Using 1912 files for training.
Found 2390 files belonging to 5 classes.
Using 478 files for validation.


2022-12-06 14:41:20.920984: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
# Fit model

history = model.fit(trainDs,
epochs=10,
validation_data=valDs)

# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


2022-12-06 14:41:41.117169: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 154140672 exceeds 10% of free system memory.


 1/60 [..............................] - ETA: 6:24 - loss: 1.9174 - acc: 0.2188

2022-12-06 14:41:41.698899: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 154140672 exceeds 10% of free system memory.


...

 3/60 [>.............................] - ETA: 31s - loss: 1.7469 - acc: 0.3333

2022-12-06 14:41:42.776867: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 154140672 exceeds 10% of free system memory.


 4/60 [=>............................] - ETA: 30s - loss: 1.6811 - acc: 0.3359

2022-12-06 14:41:43.251389: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 154140672 exceeds 10% of free system memory.


60/60 [==============================] - 42s 600ms/step - loss: 0.7719 - acc: 0.7066 - val_loss: 0.4540 - val_acc: 0.8452
Epoch 2/10
60/60 [==============================] - 33s 553ms/step - loss: 0.4541 - acc: 0.8279 - val_loss: 0.4179 - val_acc: 0.8556
Epoch 3/10
60/60 [==============================] - 30s 502ms/step - loss: 0.3730 - acc: 0.8572 - val_loss: 0.4312 - val_acc: 0.8431
Epoch 4/10
60/60 [==============================] - 32s 530ms/step - loss: 0.3311 - acc: 0.8703 - val_loss: 0.4389 - val_acc: 0.8431
Epoch 5/10
60/60 [==============================] - 33s 554ms/step - loss: 0.3145 - acc: 0.8813 - val_loss: 0.4058 - val_acc: 0.8745
Epoch 6/10
60/60 [==============================] - 34s 562ms/step - loss: 0.2880 - acc: 0.8870 - val_loss: 0.4222 - val_acc: 0.8431
Epoch 7/10
60/60 [==============================] - 33s 555ms/step - loss: 0.2587 - acc: 0.9064 - val_loss: 0.4404 - val_acc: 0.8515
Epoch 8/10
60/60 [==============================] - 33s 547ms/step - loss: 0.237

In [53]:
from PIL import Image
model.save('./saved_mode/garbage_class.h5')
# img = Image.open("/code/test_images/plastic_bottle_1.jpg").convert('L').resize((imgWidth,imgHeight), Image.ANTIALIAS)
# img = np.array(img)
# print(img.shape())

img = tf.io.read_file("/code/test_images/plastic_bottle_2.jpg")
tensor = tf.io.decode_image(img, channels=3, dtype=tf.dtypes.float32)
tensor = tf.image.resize(tensor, [imgHeight, imgWidth])
input_tensor = tf.expand_dims(tensor, axis=0)

prediction = model.predict(input_tensor)
print(prediction)
maxValue = max(prediction[0])
print("Max probability",maxValue)

# Set classes labels
classes = ['Cardboard','Glass','Metal','Paper','Plastic']

print("Predicted index",np.where(prediction[0] == maxValue)[0][0])
print("Predicted class:",classes[np.where(prediction[0] == maxValue)[0][0]])
print(sum(prediction[0]))

1/1 [==============================] - 0s 39ms/step
[[2.3769098e-03 3.5051882e-02 1.8862981e-03 2.6233474e-04 9.6042252e-01]]
Max probability 0.9604225
Predicted index 4
Predicted class: Plastic
0.999999940773705
